In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Datasets contain information about
### Students from *Maths* and *Portuguese* class.

In [ ]:
df_mat=pd.read_csv('../input/student-alcohol-consumption/student-mat.csv')
df_por=pd.read_csv('../input/student-alcohol-consumption/student-por.csv')

In [ ]:
df_mat.head()

In [ ]:
df_por.head()

## Creating dummy variables for each *categorical* column

In [ ]:
obj_ft=[]
for col in df_mat.columns:
    if(df_mat[col].dtype=='object'):
        print(f'Unique values in column {col}:',(df_mat[col].unique()))
        obj_ft.append(col)

In [ ]:
mat_dum=pd.get_dummies(df_mat[obj_ft])

In [ ]:
por_dum=pd.get_dummies(df_por[obj_ft])

# Exploring the datasets
### Diving into the distribution of data

In [ ]:
print('Unique schools with Maths subject: ',df_mat.school.unique(),'\nUnique schools with Portuguese subject: ',df_por.school.unique())

In [ ]:
# % of total students from each school in Maths class
print('% of students from GP school: ',round(len(df_mat[df_mat['school']=='GP'])/len(df_mat)*100,2),'%','\n% of students from MS school: ',round(len(df_mat[df_mat['school']=='MS'])/len(df_mat)*100,2),'%')

In [ ]:
# % of total students from each school in Portuguese class
print('% of students from GP school: ',round(len(df_por[df_por['school']=='GP'])/len(df_por)*100,2),'%','\n% of students from MS school: ',round(len(df_por[df_por['school']=='MS'])/len(df_por)*100,2),'%')

In [ ]:
sns.set_style('darkgrid')
fig,ax=plt.subplots(figsize=(8,4))
sns.countplot(data=df_mat,x='age',hue='sex')

In [ ]:
fig,ax=plt.subplots(figsize=(10,6))
sns.violinplot(data=df_por,x='reason',y='G3',hue='sex')

### Above plot shows that for any reason
#### A ***Girl*** joins the school, has good grades!

In [ ]:
fig,ax=plt.subplots(figsize=(10,6))
sns.violinplot(data=df_por,x='Fjob',y='G3',hue='sex',split=True)

### Both *Boys* & *Girls* perform well if their **Father** is a teacher

In [ ]:
fig,ax=plt.subplots(figsize=(12,4))
sns.swarmplot(data=df_por,x='Mjob',y='G3',hue='sex',dodge=True)

### If the *Mother* is in a service or at-home,
#### the *Female* child performs well!

# Preparing our final dataset
### By dropping extra columns and merging the data-frames

In [ ]:
df_mat=pd.concat([df_mat,mat_dum],axis=1)
df_por=pd.concat([df_por,por_dum],axis=1)

In [ ]:
df_mat=df_mat.drop(obj_ft,axis=1)
df_por=df_por.drop(obj_ft,axis=1)

In [ ]:
df=pd.concat([df_mat,df_por],axis=0)

# Feature-selection for regression task!

In [ ]:
from sklearn.feature_selection import mutual_info_regression
X=df.drop('G3',axis=1)
Y=df['G3']
mi_scores = mutual_info_regression(X,Y)
mi_scores = pd.Series(mi_scores, name="MI Scores",index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)

In [ ]:
print(f'{mi_scores.head(10)*100}')

## In actual scenario, only 2 columns/Features are related to **G3** score
* **G2**
* **G1**

# Splitting the dataset into:
1. Train
2. Test
3. Validation

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
Xtr,Xte,Ytr,Yte=train_test_split(df[['G2','G1']],df['G3'],random_state=108,test_size=0.27)
xtr,xte,ytrval,yteval=train_test_split(Xtr,Ytr,random_state=108,test_size=0.27)

# Building a Neural model to predict the scores!

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import callbacks,layers

In [ ]:
model=keras.Sequential([
    layers.Dense(units=264,input_shape=(2,),activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(units=264,activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(units=264,activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(1,activation='relu')
])

In [ ]:
model.compile(loss='mean_absolute_error',optimizer='adam',metrics=['mean_squared_error'])

In [ ]:
call=callbacks.EarlyStopping(patience=15,restore_best_weights=True,min_delta=0.0001)
history=model.fit(xtr,ytrval,validation_data=(xte,yteval),batch_size=30,epochs=150,callbacks=[call])

In [ ]:
training=pd.DataFrame(history.history)

In [ ]:
training.head()

In [ ]:
fig,ax=plt.subplots(figsize=(8,5))
training[['loss','val_loss']].plot(ax=ax)

In [ ]:
fig,ax=plt.subplots(figsize=(8,5))
training[['mean_squared_error','val_mean_squared_error']].plot(ax=ax)

In [ ]:
history=model.evaluate(Xte,Yte)